In [5]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score,classification_report,f1_score
# quantum
import pennylane as qml

Dataset

In [13]:
iris = load_iris()
dataset=pd.DataFrame(iris.data,columns=iris.feature_names)
dataset["target"]=iris.target
dataset

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2
146,6.3,2.5,5.0,1.9,2
147,6.5,3.0,5.2,2.0,2
148,6.2,3.4,5.4,2.3,2


In [26]:
features= dataset[dataset.columns[1:4]]
target=dataset["target"]

In [27]:
# split the data into training set and testing set 
X_train,X_test,Y_train,Y_test=train_test_split(features,target,test_size=0.25,random_state=42)

In [28]:
# Scale the data
from sklearn.preprocessing import MinMaxScaler

minmax=MinMaxScaler()
X_train=minmax.fit_transform(X_train)
X_test=minmax.transform(X_test)
X_test=np.clip(X_test,0,1)

Classical SVM

In [30]:
import time
start_time=time.time()
svm_model=SVC(kernel="rbf")
svm_model.fit(X_train, Y_train)
end_time=time.time()
print(end_time-start_time)

0.004511833190917969


In [31]:
predictions = svm_model.predict(X_test)

# Calculate accuracy and other metrics
predictions=svm_model.predict(X_test)
accuracy=accuracy_score(Y_test,predictions)
print(f"the accuracy is: {accuracy:.2f}%")
mocrof1=f1_score(Y_test,predictions,average='macro')
print(f"the mocro f1-score is:{mocrof1:.2f}%")
print(classification_report(Y_test,predictions))

the accuracy is: 1.00%
the mocro f1-score is:1.00%
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        15
           1       1.00      1.00      1.00        11
           2       1.00      1.00      1.00        12

    accuracy                           1.00        38
   macro avg       1.00      1.00      1.00        38
weighted avg       1.00      1.00      1.00        38



quantum SVM

In [32]:
num_qubits=4
device=qml.device('default.qubit',wires=num_qubits)

In [33]:
@qml.qnode(device)
def qnode(inputs,weights):
    qml.templates.AngleEmbedding(inputs, wires=range(num_qubits), rotation='Y')
    qml.adjoint(qml.AngleEmbedding(features=weights, wires=range(num_qubits), rotation='Y'))
    return qml.probs(wires=range(num_qubits))

In [34]:
def qkernel(inputs,weights):
    return np.array([[qnode(input,weight)[0] for weight in weights] for input in inputs ])

In [35]:
# Fit the model
start_time=time.time()
qsvm=SVC(kernel=qkernel)
qsvm.fit(X_train,Y_train)
end_time=time.time()
print(end_time-start_time)

74.31547856330872


In [36]:
# Make predictions and calculate the accuracy
predictions=qsvm.predict(X_test)
accuracy=accuracy_score(Y_test,predictions)
print(f"the accuracy is: {accuracy:.2f} %")
macrof1=f1_score(Y_test,predictions,average='macro')
print(f"the mocro f1-score is: {mocrof1:.2f}%")
print(classification_report(Y_test,predictions))

the accuracy is: 1.00 %
the mocro f1-score is: 1.00%
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        15
           1       1.00      1.00      1.00        11
           2       1.00      1.00      1.00        12

    accuracy                           1.00        38
   macro avg       1.00      1.00      1.00        38
weighted avg       1.00      1.00      1.00        38

